In [11]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

In [2]:
# 1. Embedding-Modell vorbereiten
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [3]:
# 2. Datei laden
loader = TextLoader("../knowledge_base/ChinookData.json", encoding='utf-8')
documents = loader.load()

In [4]:
# 3. Text in Chunks aufteilen
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

In [5]:

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [6]:
# 5. Dokumente hinzufügen und speichern
vector_store.add_documents(texts)

['ce1ea10f-ea11-43cc-92f8-3b78df59eafc']

In [10]:
# LLM über Ollama
llm = Ollama(model="mistral")

C:\Users\meric\AppData\Local\Temp\ipykernel_2400\889653991.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")


In [13]:
# Retriever aus Chroma
retriever = vector_store.as_retriever()

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [16]:
frage = "Kannst du mir ein Linkml schema das die daten bzws das format sowie die relationen wiederspiegelt?"
antwort = rag_chain({"query": frage})

print("Antwort:", antwort["result"])
print("Quellen:", [doc.metadata for doc in antwort["source_documents"]])

Antwort:  Hier ist ein Beispiel für ein LinkML Schema, das die angegebene Datenstruktur wiedergeben sollte. Das LinkML Schema definiert die Klasse `Song` mit den Eigenschaften `id`, `title`, und `album`. Zusätzlich wird eine Relation `belongsToAlbum` definiert, um zu sagen, dass jede Komposition einem Album zugeordnet ist.

```yaml
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix lmm: <https://w3id.org/linkml/lmm> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

<Song> rdf:Class ;
    lmm:hasProperty [ a rdf:Property ;
                        lmm:propertyType xsd:string ;
                        lmm:propertyName "id" ;
                        lmm:propertyDocumentation "The unique identifier of the song." ] ,
    lmm:hasProperty [ a rdf:Property ;
                        lmm:propertyType xsd:string ;
               